In [1]:
import torch
import numpy as np
from skimage.color import rgb2lab
from skimage.io import imread

In [2]:
def compute_morans_i_torch(channel):
    """
    Compute Moran's I for a single-channel image using PyTorch.
    Args:
        channel (torch.Tensor): 2D tensor representing a single color channel.
    Returns:
        float: Moran's I value.
    """
    # Flatten the channel and compute mean
    N = channel.numel()
    mean_value = torch.mean(channel)

    # Create spatial weight matrix for 4-nearest neighbors
    height, width = channel.shape
    weights = torch.zeros((height, width), device=channel.device)

    # Shift the channel to simulate neighboring relationships
    if height > 1:
        weights[:-1, :] += channel[1:, :]  # Up
        weights[1:, :] += channel[:-1, :]  # Down
    if width > 1:
        weights[:, :-1] += channel[:, 1:]  # Right
        weights[:, 1:] += channel[:, :-1]  # Left

    # Numerator: weighted sum of deviations
    deviations = (channel - mean_value)
    numerator = torch.sum(weights * deviations * deviations)

    # Denominator: sum of squared deviations
    denominator = torch.sum(deviations ** 2)

    # Normalization factor
    w_sum = torch.sum(weights)
    morans_i = (N / w_sum) * (numerator / denominator)

    return morans_i.item()


In [3]:

# Load and preprocess the image
def preprocess_image(image_path):
    """
    Load an image and convert it to LAB space.
    Args:
        image_path (str): Path to the image.
    Returns:
        tuple: LAB channels as PyTorch tensors (L, A, B).
    """
    # Load image and convert to LAB
    image = imread(image_path) / 255.0  # Normalize to [0, 1]
    lab_image = rgb2lab(image)

    # Extract LAB channels and convert to PyTorch tensors
    L = torch.tensor(lab_image[:, :, 0], dtype=torch.float32)
    A = torch.tensor(lab_image[:, :, 1], dtype=torch.float32)
    B = torch.tensor(lab_image[:, :, 2], dtype=torch.float32)

    return L, A, B




In [6]:
# Path to the image
image_path = "/home/oem/eliza/DL/project/data/artifact/generated/photo2monet/img010694.jpg"

# Preprocess the image
L, A, B = preprocess_image(image_path)

# Compute Moran's I for each channel
morans_L = compute_morans_i_torch(L)
morans_A = compute_morans_i_torch(A)
morans_B = compute_morans_i_torch(B)

# Print results
print(f"Moran's I for L (Lightness): {morans_L}")
print(f"Moran's I for A (Red-Green): {morans_A}")
print(f"Moran's I for B (Blue-Yellow): {morans_B}")

Moran's I for L (Lightness): 1.0271379947662354
Moran's I for A (Red-Green): 0.9829598069190979
Moran's I for B (Blue-Yellow): 1.1065735816955566
